In [36]:
import sys
sys.path.insert(0, '../../')
from questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from questions_construction.domains import DOMAIN_NAMES

from analysis.model_performances import *
from common import *
import random

In [98]:
model_name = 'gpt-4o' #'gemini' #
by_file_name = {}
for i in range(1,4):
    # try:
    with open(f'{PROJECT_PATH}/results.human_eval.free_answers/{model_name}/human_{i}.all.{model_name}.json') as f:
        data_id = f'human_{i}'
        data = json.load(f)
        by_file_name[data_id] = {d['id']: d for _category, d_ls in data.items() for d in d_ls}
        for _id, d in by_file_name[data_id].items():
            d['annotator'] = data_id
    # except:
    #     print(f'{model_name}.human_{i}')

In [99]:
ids_by_category = defaultdict(set)
for file_name, d_ls in by_file_name.items():
    for _id, d in d_ls.items():
        ids_by_category[d['question_category']].add(d['id'])

In [100]:
results_by_category = {}

# category_name = 'object_tracking'
total_scores = []
scores_all_by_annotator = defaultdict(list)
for category_name in QUESTION_CATEGORIES:
    category_scores = []
    for q_id in ids_by_category[category_name]:
        annotators_scores = []
        for annotator, data_by_id in by_file_name.items():
            score = int(data_by_id[q_id]['human_score'])
            annotators_scores.append(score)
            scores_all_by_annotator[annotator].append(score)
        mean_score = np.mean(annotators_scores)
        if mean_score > 0.5:
            category_scores.append(1)
        else:
            category_scores.append(0)
    total_scores += category_scores
    mean = np.mean(category_scores)
    std = np.std(category_scores)
    sem = std / np.sqrt(len(category_scores))
    results_by_category[category_name] = {'mean': mean, 'std': std, 'sem': sem, 'scores': category_scores, 'n': len(category_scores)}
    
mean_total = np.mean(total_scores)
std_total = np.std(total_scores)
sem_total = std_total / np.sqrt(len(total_scores))
print(f'Total mean: {mean}, std: {std}, sem: {sem}')

Total mean: 0.5, std: 0.5, sem: 0.11180339887498948


In [101]:
for category_name, results in results_by_category.items():
    print(f'{category_name}: '+'${'+ f'{results["mean"]*100}'+'}_{'+ f'{round(results["sem"]*100, 2)}'+'}$')
print(f'TOTAL AVG: '+'${'+ f'{mean_total*100}'+'}_{'+ f'{round(sem_total*100, 2)}'+'}$')

object_tracking: ${85.0}_{7.98}$
fluent_tracking: ${30.0}_{10.25}$
state_tracking: ${45.0}_{11.12}$
action_executability: ${45.0}_{11.12}$
effects: ${30.0}_{10.25}$
numerical_reasoning: ${15.0}_{7.98}$
hallucination: ${40.0}_{10.95}$
composite: ${50.0}_{11.18}$
TOTAL AVG: ${42.5}_{3.91}$


# Proportionate Agreements

In [102]:
from sklearn.metrics import cohen_kappa_score

kappa_scores = []
kappa_scores.append(cohen_kappa_score(scores_all_by_annotator[f'human_1'], scores_all_by_annotator[f'human_2']))
kappa_scores.append(cohen_kappa_score(scores_all_by_annotator[f'human_1'], scores_all_by_annotator[f'human_3']))
kappa_scores.append(cohen_kappa_score(scores_all_by_annotator[f'human_2'], scores_all_by_annotator[f'human_3']))
kappa_scores
kappa_scores_avg = np.mean(kappa_scores)
kappa_scores_avg

0.7379448746887349

In [103]:
kappa_scores

[0.7653194263363755, 0.711508309814989, 0.7370068879148404]